# Publication Plots for the Sherrington-Kirkpatrick Model

In [ ]:
using HDF5, Printf
using LsqFit, Measurements
using Statistics, Distributions

using PyCall, PyPlot
np = pyimport("numpy")
h5py = pyimport("h5py")
PyPlot.plt.style.use("paper.mplstyle")

In [ ]:
# path to folder with the h5 files
PATH = "../data/SK_model/"

## Plots

### Average Energy

In [ ]:
E_0 = -0.763
p = 1000
num_instances = 10000
seed = 137

E_star_data = []
E_star_std_data = []
# for num in vcat([10 + k * 5 for k in 0:22], [200])
for num in vcat([10 + k * 10 for k in 0:11], [150, 180, 200])
    N = num
    new_file = "SK_model_" * "p_" * string(p) * "_N_" * string(N) * "_num_inst_" * string(num_instances) * "_seed_" * string(seed) * "_moments.h5"
    data_file = h5open(PATH * new_file, "r")
    println(new_file)
    E_star = read(data_file, "E_star")[1]
    E_star_squared = read(data_file, "E_star_squared")[1]
    push!(E_star_data, E_star / N - E_0)
    push!(E_star_std_data, sqrt.(E_star_squared - E_star^2) / N)
end

In [ ]:
# N = vcat([10 + k * 5 for k in 0:22], [200])
N = vcat([10 + k * 10 for k in 0:11], [150, 180, 200]);

In [ ]:
m(x, p) = p[1] .+ p[2] .* x
p0 = [0., -0.5];

In [ ]:
fit_E = curve_fit(m, log.(N), log.(E_star_data), p0)    
println(fit_E.resid .|> abs |> sum)    
println(fit_E.param)

In [ ]:
fit_σ = curve_fit(m, log.(N), log.((E_star_std_data)), p0)    
println(fit_σ.resid .|> abs |> sum)    
println(fit_σ.param)

In [ ]:
fig = figure(figsize=(4.2, 3))
ax = subplot(211)
axhline(0.053, c="k", ls="--")
ax.loglog(N, E_star_data, "o")
ax.loglog(N, map(x -> m(x, [fit_E.param[1], fit_E.param[2]]), log.(N)) .|> exp, "-C0", label="\$N^{-\\omega}\$")
ax.loglog([], [], lw=0, label=@sprintf("\$ \\omega = %.2f \$", fit_E.param[2] |> abs))
ax.set_ylabel("\$\\langle E_*\\rangle / N - \\varepsilon_P\$")
ax.set_xticklabels([])
ax.set_xlim(10, 200)
ax.set_ylim(0.01, 0.5)
ax.legend(ncol=2, columnspacing=0.1)

ax = subplot(212)
ax.loglog(N, E_star_std_data, "o")
ax.loglog(N, map(x -> m(x, [fit_σ.param[1], fit_σ.param[2]]), log.(N)) .|> exp, "-C0", label="\$N^{-\\omega_s}\$")
ax.loglog([], [], lw=0, label=@sprintf("\$ \\omega_s = %.2f \$", fit_σ.param[2] |> abs))
ax.set_ylim(0.01, 0.5)
ax.set_xlim(10, 200)
ax.set_xlabel("\$N\$")
ax.set_ylabel("\$ s / N\$")
ax.legend(ncol=2, columnspacing=0.1)

tight_layout(pad=0.15, w_pad=0.0, h_pad=0.0)
# savefig("../plots/" * "Fig1.pdf", dpi=300)

### Histograms

In [ ]:
N = 20;

#### Mean-field statistics

In [ ]:
new_file = "SK_model_" * "p_" * string(p) * "_N_" * string(N) * "_num_inst_" * string(num_instances) * "_seed_" * string(seed) * "_hist_stats.h5"
data_file = h5open(PATH * new_file, "r")
all_E_stars = read(data_file, "all_E_stars")
counts, bins = np.histogram((all_E_stars .- mean(all_E_stars)) ./ sqrt(var(all_E_stars)), bins=20)

#### Exact data

In [ ]:
new_file = "SK_model" * "_N_" * string(N) * "_num_inst_" * string(num_instances) * "_seed_" * string(seed) * "_exact.h5"
data_file = h5open(PATH * new_file, "r")
all_E_0s = []
for i in 1:num_instances
    E_0 = read(data_file, "set_" * string(i) * "/E_0")
    push!(all_E_0s, E_0)
end

counts_0, bins_0 = np.histogram((all_E_0s .- mean(all_E_0s)) ./ sqrt(var(all_E_0s)), bins=20)

#### Plots

In [ ]:
g_m(x, m, u, v, w) = w .* exp.(m .* (x .- u) ./ v - m .* exp.((x .- u) ./ v))

In [ ]:
X = np.linspace(-6, 5, 101)

fig = figure(figsize=(4.2, 3))
ax = subplot(111)
plot(X, map(x -> g_m(x, 6, 0.2, 2.35, 7e5), X), label="Gumbel", "--C3", lw=1.5)
hist(bins[1:end-1], bins, weights=(counts), alpha=1.0, lw=1.5, color="k", label="mean-field", histtype="step")
hist(bins_0[1:end-1], bins_0, weights=(counts_0), alpha=0.75, label="exact", color="grey")#, histtype="step")#, hatch="x", edgecolor="w")
legend(loc="upper left", ncol=1)
ax.set_yscale("log")
ax.set_xlim(-6, 4)
ax.set_ylim(1e0, 1e4)
ax.set_xticks([-6 + 2k for k in 0:5])
ax.set_xlabel("\$\\left(E - \\langle E \\rangle\\right) / s\$")
ax.set_ylabel("\$ P(E)\$")
tight_layout(pad=0.15, w_pad=0.0, h_pad=0.0)
# savefig("../plots/" * "Fig2.pdf", dpi=300)

#### Tails

In [ ]:
tail_counts, tail_bins = np.histogram((all_E_stars .- all_E_0s) ./ abs(mean(all_E_0s)), bins=100);

In [ ]:
fig = figure(figsize=(4.2, 2.5))
ax = subplot(111)
hist(tail_bins[1:end-1], tail_bins, weights=tail_counts, alpha=0.75, lw=1.5, color="grey")
ax.set_xlabel("\$\\varepsilon_* = \\left(E_* - E_0\\right) / |\\langle E_0 \\rangle| \$")
ax.set_ylabel("\$ P(\\varepsilon_*)\$")
ax.set_yscale("log")
# axvline(cutoff, c="C3", label="cutoff")
# legend(loc="upper right", ncol=2)
ax.set_xlim(0, 0.2)
ax.set_ylim(0., 1e4)
ax.set_yticks([1e1, 1e2, 1e3, 1e4])
tight_layout(pad=0.15, w_pad=0.0, h_pad=0.0)
# savefig("../plots/" * "Fig3.pdf", dpi=300)

In [ ]:
tails = []
for i in 1:size(tail_bins)[1]
    push!(tails, sum(tail_counts[i:end]) ./ num_instances)
end

In [ ]:
N_range = 5:20;

In [ ]:
all_tail_counts = Dict()
all_tail_bins = Dict()
for N in N_range
    new_file = "SK_model_" * "p_" * string(p) * "_N_" * string(N) * "_num_inst_" * string(num_instances) * "_seed_" * string(seed) * "_hist_stats.h5"
    data_file = h5open(PATH * new_file, "r")
    all_E_stars = read(data_file, "all_E_stars")

    new_file = "SK_model" * "_N_" * string(N) * "_num_inst_" * string(num_instances) * "_seed_" * string(seed) * "_exact.h5"
    data_file = h5open(PATH * new_file, "r")
    all_E_0s = []
    for i in 1:num_instances
        E_0 = read(data_file, "set_" * string(i) * "/E_0")
        push!(all_E_0s, E_0)
    end

    tail_counts, tail_bins = np.histogram((all_E_stars .- all_E_0s) ./ abs(mean(all_E_0s)), bins=100)
    tail_bins[1] = 1e-10
    all_tail_counts[N] = tail_counts
    all_tail_bins[N] = tail_bins
end

In [ ]:
cutoff = 0.1;

In [ ]:
all_tails = Dict()
all_tail_fits = Dict()
all_tail_ycuts = Dict()
for N in N_range
    tails = []
    for i in 1:size(all_tail_bins[N])[1]
        push!(tails, sum(all_tail_counts[N][i:end]) ./ num_instances)
    end
    all_tails[N] = tails .+ 1e-15
    cutoff_idx = findfirst(y -> y == argmin(x -> abs.(x .- cutoff), all_tail_bins[N]), all_tail_bins[N])
    tail_fit = curve_fit(m, all_tail_bins[N][2:end-cutoff_idx], log.(all_tails[N][2:end-cutoff_idx]), [1., -10])
    all_tail_ycuts[N] = tail_fit.param[1] 
    all_tail_fits[N] = tail_fit.param[2]  
end

In [ ]:
fig = figure(figsize=(4.5, 3))

styles = ["^", "o", "s", "v"]
colors = ["#000C8F", "grey", "#0A92C4"]
ax = subplot(111)
for (k, N) in enumerate([10, 15, 20])
    cutoff_idx = findfirst(y -> y == argmin(x -> abs.(x .- cutoff), all_tail_bins[N]), all_tail_bins[N])
    
    if k == 1
        ax.semilogy(sqrt(N) .* all_tail_bins[N][2:cutoff_idx], map(x -> exp(-2pi*x + 2pi * sqrt(N) * all_tail_bins[N][2] + log(all_tails[N][2])), sqrt(N) .* all_tail_bins[N][2:cutoff_idx]), 
            label="\$\\mathrm{e}^{-2\\pi\\sqrt{N}\\varepsilon}\$", "-k")
    else
        ax.semilogy(sqrt(N) .* all_tail_bins[N][2:cutoff_idx], map(x -> exp(-2pi*x + 2pi * sqrt(N) * all_tail_bins[N][2] + log(all_tails[N][2])), sqrt(N) .* all_tail_bins[N][2:cutoff_idx]), 
            "-k")
    end     
    
    ax.semilogy(sqrt(N) .* all_tail_bins[N][1:cutoff_idx], all_tails[N][1:cutoff_idx] , styles[k], c=colors[k], label=@sprintf("\$N=%2d\$", N), ms=5)
   
end
ax.set_xlim(0, 0.4)
ax.set_ylim(1e-2, 3e-1)
ax.set_xlabel("\$  \\sqrt{N}\\varepsilon \$")
ax.set_ylabel("\$ P_f(\\varepsilon_* > \\varepsilon) \$")
legend(loc="lower left", ncol=1, handlelength=1.0)

tight_layout(pad=0.4)
# savefig("../plots/" * "Fig4.pdf", dpi=300)

### Fluctuations

#### Easy instance

In [ ]:
PATH_DB = PATH * "Easy_Instance_N11/"
FILE = "Easy_instance_SK_model_seed_61_N_11_tau_05_p_1000.h5"
filename = PATH_DB * FILE
h5file = h5open(filename, "r")
E_0 = read(h5file, "E0")
J = read(h5file, "J_ij")
exact_levels = read(h5file, "Levels")
mf_level = read(h5file, "MF_Energy-E0")
lyapunov = read(h5file, "Lyapunov_Exp");
# omega = read(h5file, "omega_n");

In [ ]:
times = np.linspace(0, 1, size(E_0)[1]);
times_medium = np.linspace(0, 1, size(lyapunov)[1]);

#### Hard instance

In [ ]:
PATH_DB = PATH * "Hard_Instance_N11/"
FILE = "Hard_instance_SK_model_seed_2041_N_11_tau_05_p_5000.h5"
filename = PATH_DB * FILE
h5file = h5open(filename, "r")
E_0 = read(h5file, "E0")
J = read(h5file, "J_ij")
exact_levels_hard = read(h5file, "Levels")
mf_level_hard = read(h5file, "MF_Energy-E0")
lyapunov_hard = read(h5file, "Lyapunov_Exp")
omega_hard = read(h5file, "omega_n");

In [ ]:
times_long = np.linspace(0, 1, size(lyapunov_hard)[1]);

#### Combined Plot

In [ ]:
fig = figure(figsize=(4.5,  3.6))
ax = subplot(221)
for k in 1:20
    if k == 1
        ax.plot(times, exact_levels[:, k], "-C0", label="Exact", lw=0.75)
    else
        ax.plot(times, exact_levels[:, k], "-C0", lw=0.75)
    end
end

ax.plot(times, mf_level, "--C1", lw=2, label="Mean-field")
ax.set_ylabel("\$ E - E_0 \$")
ax.set_xticklabels([])
ax.set_xlim(0, 1)
ax.set_ylim(0, 4)

ax = subplot(223)
ax.plot(times_medium, lyapunov[:, 1], "-C3")
ax.plot(times_medium, lyapunov[:, 2], "-C7")
ax.plot(times_medium, lyapunov[:, 3], "-C7")
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_yticks([0, 0.5, 1])
ax.set_xlabel("\$s\$")
ax.set_ylabel("\$ \\lambda_i \$")

# =================================================

ax = subplot(222)
for k in 1:20
    if k == 1
        ax.plot(times, exact_levels_hard[:, k], "-C0", label="Exact", lw=0.75)
    else
        ax.plot(times, exact_levels_hard[:, k], "-C0", lw=0.75)
    end
end

ax.plot(times, mf_level_hard, "--C1", lw=2, label="Mean-field")

ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_xlim(0, 1)
ax.set_ylim(0, 4)
ax.legend(loc="upper right", handlelength=1.3, markerfirst=false)

ax = subplot(224)
ax.plot(times_long, lyapunov_hard[:, 1], "-C3")
ax.plot(times_long, lyapunov_hard[:, 2], "-C7")
ax.plot(times_long, lyapunov_hard[:, 3], "-C7")
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_yticks([0, 0.5, 1])
ax.set_yticklabels([])
ax.set_xlabel("\$s\$")

tight_layout(pad=0.15, w_pad=0.5, h_pad=0.0)
# savefig("../plots/" * "Fig7.pdf", dpi=300)

#### Hard instance (large)

In [ ]:
PATH_DB = PATH * "Hard_Instance_N18/"
FILE = "Hard_instance_SK_model_N_18_tau_05_p_5000.h5"
filename = PATH_DB * FILE
h5file = h5open(filename, "r")
lyapunov_hard_18_5000 = read(h5file, "Lyapunov_Exp")
omega_hard_18 = read(h5file, "omega_n");

In [ ]:
PATH_DB = PATH * "Hard_Instance_N18/"
FILE = "Hard_instance_SK_model_N_18_tau_05_p_20000.h5"
filename = PATH_DB * FILE
h5file = h5open(filename, "r")
exact_levels_hard_18 = read(h5file, "Levels")
mf_level_hard_18 = read(h5file, "MF_Energy-E0")
lyapunov_hard_18 = read(h5file, "Lyapunov_Exp")
omega_hard_18 = read(h5file, "omega_n");

In [ ]:
PATH_DB = PATH * "Hard_Instance_N33/"
FILE = "Hard_instance_SK_model_N_33_tau_04_p_20000.h5"
filename = PATH_DB * FILE
h5file = h5open(filename, "r")
lyapunov_hard_33 = read(h5file, "Lyapunov_Exp")
omega_hard_33 = read(h5file, "omega_n");

In [ ]:
fig = figure(figsize=(4.5,  3.6))
grid = PyPlot.plt.GridSpec(2, 2)

ax = subplot(get(grid, (0, 0)))
times = np.linspace(0, 1, size(mf_level_hard_18)[1]);
times_long = np.linspace(0, 1, size(lyapunov_hard_18)[1]);

for k in 1:30
    if k == 1
        ax.plot(times, exact_levels_hard_18[:, k], "-C0", label="Exact", lw=0.75)
    else
        ax.plot(times, exact_levels_hard_18[:, k], "-C0", lw=0.75)
    end
end

ax.plot(times, mf_level_hard_18, "--C1", lw=2, label="Mean-field")
ax.set_ylabel("\$ E - E_0 \$")
ax.set_xticklabels([])
ax.set_xlim(0, 1)
ax.set_ylim(0, 4)
ax.legend(loc="upper center", handlelength=1.25, markerfirst=false)


ax = subplot(get(grid, (1, 0)))

ax.plot(np.linspace(0, 1, size(lyapunov_hard_18_5000)[1]), lyapunov_hard_18_5000[:, 1], "-k")
ax.plot(times_long, lyapunov_hard_18[:, 1], "-C3")
ax.plot(times_long, lyapunov_hard_18[:, 2], "-C7")
ax.plot(times_long, lyapunov_hard_18[:, 3], "-C7")
ax.plot([], [], label="\$N = 18\$", lw=0)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_yticks([0, 0.5, 1])
ax.set_xlabel("\$s\$")
ax.set_ylabel("\$ \\lambda_i \$")
ax.legend(loc="upper left", handlelength=0)


ax = subplot(get(grid, (pycall(pybuiltin("slice"), PyObject, 0, 2), 1)))
ax.plot(times_long, lyapunov_hard_33[:, 1], "-C3")
ax.plot(times_long, lyapunov_hard_33[:, 2], "-C7")
ax.plot(times_long, lyapunov_hard_33[:, 3], "-C7")
ax.plot([], [], label="\$N = 33\$", lw=0)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1.5)
ax.set_yticks([k * 0.5 for k in 0:3])
ax.set_xlabel("\$s\$")
ax.set_ylabel("\$ \\lambda_i \$", labelpad=25)
ax.yaxis.set_label_position("right")
ax.yaxis.tick_right()
ax2 = ax.twinx()
ax2.set_ylim(0, 1.5)
ax2.set_yticks([k * 0.5 for k in 0:3])
ax.set_yticklabels([])
ax.legend(loc="upper left", handlelength=0)

tight_layout(pad=0.15, w_pad=0.5, h_pad=0.0)
# savefig("../plots/" * "Fig8.pdf", dpi=300)